# LT Toolbox: Application of Trajectory Arrays to MITgcm trajectories evaluated in the Irminger Sea

Welcome to the Lagrangian Trajectories Toolbox tutorial! 

The LT Toolbox is a Python library dedicated to the post-processing, visualisation and analysis of Lagrangian water parcel trajectories. The toolbox offers users two structures for working with Lagrangian trajectories: Trajectory Arrays (TrajArrays) and Trajectory Frames (TrajFrames). In this tutorial, we explore TrajArrays, which make use of [xarray](http://xarray.pydata.org/en/stable/#) multidimensional DataArrays to store attribute variables associated with trajectories (e.g. lat, lon, in-situ temperature etc.).

In this tutorial, we will show how to:

+ **Store** the output of an example collection of MITgcm Lagrangian trajectory code in a TrajArray object.

+ **Add** new variables, such as particle IDs and seeding levels, to your dataset.

+ **Filter** trajectories using any attribute variable contained in your dataset.

+ **Get** existing features, including trajectory start/end times, start/end locations and durations.

+ **Compute** metrics, such as distance travelled, particle displacements, velocities and Lagrangian probabilities from trajectories.

+ **Plot** trajectory data in the form of time series, temperature-salinity diagrams and more.

+ **Map** trajectories, properties and probability distributions onto the Earth's surface using [Cartopy](https://tracmass.readthedocs.io/en/latest/).

## Getting Started

Let us begin by importing the relevant packages we'll need to get started with the LT Toolbox. 

**Note**: Since lt_toolbox is still undergoing unit testing, the package is not yet available on PyPi, hence we use a local development version.

In [ ]:
import sys
import xarray as xr
import numpy as np
import matplotlib.pyplot as plt

# Navigating to the local development version of lt_toolbox.
sys.path.append("/home/snapdragon/HadGEM3-GC31-MM/Proj_Future_Pathways/src/Software/lt_toolbox/")

import lt_toolbox as ltt

### Storing Trajectory Data

To explore the functionality of the LT Toolbox, we will use output from the example MITgcm simulation of Lagrangian trajectories advected forwards-in-time from the northward inflows to the upper 1000 m of the Irminger Sea.

MITgcm trajectory output is provided in .mat files, so we first use Export_MITgcm_Lagrangian_trajectories_to_zarr.ipynb to reformat the output data into a .nc file conforming to the standard NCEI trajectory [template](https://www.nodc.noaa.gov/data/formats/netcdf/v2.0/trajectoryIncomplete.cdl).

Below we load the resulting .nc file as a DataSet with xarray, before creating a TrajArray, traj.

In [ ]:
# Navigate to the directory where the ORCA1 output data is stored.
trajDir = "/home/snapdragon/HadGEM3-GC31-MM/Proj_Future_Pathways/src/Software/lt_toolbox/tutorials/data/"
trajFilename = "MITgcm_2km_upper_1000m_example_traj.zarr"

# Open output .nc file as a DataSet.
dataset = xr.open_zarr(trajDir + trajFilename, chunks=None)
# Modify time variable to be in hours from day fraction:
dataset['time'].values = dataset['time'].values * (24)

# Create a TrajArray from the DataSet.
traj = ltt.TrajArray(dataset)

**What is a TrajArray?**

The TrajArray, traj, hosts the original xarray DataSet accessible as an attribute with traj.data. 

For improved functionality, attribute variables (data variables stored in our DataSet) are made accessible with traj.{variable} to access a given {variable}.

The true value of a trajectories object comes with the use of the built-in functions specifically designed for post-processing, visualisation and analysis of Lagrangain water parcel trajectories.

### Exploring our TrajArray

By accessing the .data attribute of our TrajArray above, we can see that attribute variables are formatted with dimensions **traj** (trajectory) and **obs** (observation - represents one time-level).

**Note**: Since trajectories are of differing lengths, missing **obs** values for a given **traj** are filled as NaNs or NaTs (time).

In [ ]:
# To return details of our original DataSet.
print(traj)

# To access the temp attribute variable.
print(traj.temp)

# Using datetime64 format for time instead of timedelta64.
# Start date of simulation is 2007-01-01
traj = traj.use_datetime('2007-01-01', 'h')


### Adding new attribute variables to our TrajArray.

To add a new attribute variable to a TrajArray, including appending it to the original DataSet, use the **.add_variable()** method. 

Two common attribute variables which a user may wish to add to their TrajArray, a unique trajectory id and the seeding level when a particles are released are included as seperate methods: **.add_id()** and **.add_seed()**.

In [ ]:
# Suppose we would like to add the id and seeding level of all of our
# particles for future analysis. 

# We can combine methods on a single line to return both.
traj = traj.add_id().add_seed()

# Let's look at our new attribute variables.
print(traj.id)
print(traj.seed_level)

In [ ]:
# Consider now if we wanted to create a new attribute variable, temp_K,
# to store the in-situ temperature in Kelvin. 

# Using .values to access the numpy array storing the values of temp.
temp_k = traj.temp.values + 273.15

# Dictionary of attributes of our new variable.
attrs = {'long_name': 'in-situ temperature in Kelvin',
         'standard_name': 'temp_K',
         'units': 'Kelvin'
        }

# Add temp_k as a new attribute variable to our trajectories object.
traj = traj.add_variable(temp_k, attrs)

# Let's see if temp_K has been added!
print(traj.temp_K)

### Filtering trajectories using an attribute variable.

Filtering our trajectories to include only those complete trajectories where a specific criteria is met is integral to Lagrangian analysis. 

The LT Toolbox includes two fully vectorised methods **.filter_between()** and **filter_equal()** to allow users to filter on any attribute variable conatined in their TrajArray.

In [ ]:
# Filtering all our trajectories to include only those released in 
# seeding level 1 and storing as a new trajectories object, traj_seed1.
traj_seed1 = traj.filter(query='seed_level == 1', drop=False)

# Let's look at traj_seed1 - 880 particles were
# released in seeding level 1.
print(traj_seed1)

In [ ]:
# Filtering all our trajectories to include only those which travel 
# between 63 N - 68 N and storing as a new TrajArray, 
# traj_sublat.
traj_sublat = traj.filter_between('lat', 63, 69, drop=False)

# Let's look at traj_sublat - only 560 particles were
# found between 63-69 N at any time during their trajectories.
print(traj_sublat)

In [ ]:
# Filtering all of our trajectories according to the unique IDs
# contained in a given list:
print(traj.filter_isin('id', [10, 20, 30, 55, 98, 129], drop=False))

### Filtering trajectories by time - A Special Case

It should be noted that when filter methods are called with a time attribute variable, rather than return complete trajectories, only the observations (**obs**) meeting the specified criteria are returned. 

In [ ]:
# Defining the min and maximum time levels to return obs for.
tmin = '2007-01-31'
tmax = '2007-03-31'

# Filtering between tmin and tmax, the values tmin and tmax are
# included, and storing as a new TrajArray, traj_subtime.
traj_subtime = traj.filter_between('time', tmin, tmax)

# Let's look at traj_subtime - only 237 obs are 
# included as expected.
print(traj_subtime)

### Filtering trajectories which intersect a polygon.

In [ ]:
# Defining square as a list of (Lat, Lon) tuples storing the coordinates
# of the polygon boundary.
square = [[[-47, 58], [-47, 63], [-40, 63], [-40, 58], [-47, 58]]]

# Filtering all our trajectories to include only those which, at some 
# point in their life time intersect a simple polygon, square.
traj_poly = traj.filter_polygon(polygon=square, method='pos', drop=False)

# Let's look at the resulting TrajArray.
print(traj_poly)

In [ ]:
# Let's look at a map of the resulting trajectories. 
traj_poly.map_trajectories()

### Finding trajectory points that intersect a polygon.

In [ ]:
# Finding trajectory points which intersect a simple polygon, square.
ind_poly = traj.find_polygon(square)

# Let's look at the resulting indices returned in ind_poly.
ind_poly

### Getting existing features from a TrajArray.

The LT Toolbox includes a range of .get_ methods to extract important features from existing attribute variables in a TrajArray. 

In [ ]:
# Get the times and locations when particle are released. 
traj = traj.get_start_time().get_start_loc()

# Get the times and locations when particles are terminated.
traj = traj.get_end_time().get_end_loc()


In [ ]:
# Get the maximum value of the temp variable for each trajectory. 
traj = traj.get_max('temp')

# Get the minimum value of the temp variable for each trajectory. 
traj = traj.get_min('temp')

# Get the value of the temp variable for each trajectory on 2007-06-01. 
traj = traj.get_value('temp', '2007-06-01')


In [ ]:
# Get the duration of each trajectory, t_total. 
traj = traj.get_duration()

# Let's look at traj.
print(traj)

### Computing diagnostic metrics for trajectories.

Computation is a further important feature of the toolbox.

In particular, there are .compute methods available to compute the distance travelled by particles along their trajectories, particle displacements (zonal/meridional/vertical) and Lagrangian velocities (zonal/meridional/vertical).

Below we show how to combine **filter**, **compute** and **plot** methods to efficiently generate visualisations of output data.

In [ ]:
# Creating a time series plot of the meridional displacement travelled 
# by the first 10 particles in our ORCA1 output data.
traj.filter_between('id', 0, 4).compute_dy().plot_timeseries('dy')

In [ ]:
# Creating a time series plot of the meridional velocity travelled by 
# the first particle in our MITgcm output data.
traj.filter_between('id', 0, 2).compute_v().plot_timeseries('v')

In [ ]:
# Plotting the corresponding trajectory coloured by along-stream potential temp:
traj.filter_between('id', 0, 2).map_trajectories(col_variable='temp')

### Computing Lagrangian probability distributions.

In [ ]:
# Computing Lagrangian probability distribution for all trajectories
# released in seed_level 1 using 1x1 degree bins.
traj_prob = traj.filter('seed_level == 1', drop=False).compute_probability(bin_res=0.25, method='traj')

# Let's look at traj_prob.
print(traj_prob)

In [ ]:
# Computing Lagrangian probability distribution for all trajectories
# released in seed_levels 1 to 3 using 1x1 degree bins. 
traj_probs = traj.filter_between('seed_level', 1, 3).compute_probability(bin_res=0.25, method='pos')

# Here the group_by function ensures that three seperate probability
# distributions are computed.
print(traj_probs)

### Further plotting of seawater properties.

Another useful plot for examining the water mass properties of a given trajectory is the temperature-salinity (t-s) diagram. 

We can very easily produce a t-s diagram with the LT Toolbox using the **.plot_ts_diagram** method.

In [ ]:
# Creating a T-S plot for the particles initialised on 2007-01-01
# and colouring by the longitude of initialisation.
traj.filter('time == 2007-01-01').plot_ts_diagram(col_variable='lon')

In [ ]:
# Plotting temperature of the particles initialised on
# 2007-01-01 (seeding level = 1) in longitude-depth space.
traj.plot_variable('temp', 'xz', 1, '2007-01-01')

### Mapping trajectories and probability distributions with Cartopy.

The LT Toolbox utilises the highly adaptable Cartopy geospatial visualisation package to map trajectories and their properties on the Earth's surface. 

Below we show how to use **.map_trajectories**, **.map_probability()** and **.map_property()** methods when anaysing trajectory output data.

In [ ]:
# Creating a map of the trajectories of the first 20 particles released,
# coloured by the in-situ temperature.
traj.filter_between('id', 0, 5).map_trajectories(col_variable='temp')

In [ ]:
# Creating a map of the binned probability of particle pathways.
traj.map_probability(bin_res=0.25, prob_type='traj', cmap='viridis')

In [ ]:
# Creating a map of the binned probability of particle positions.
traj.map_probability(bin_res=0.25, prob_type='pos', cmap='viridis')

In [ ]:
# Creating a map of the binned mean temperature of particles.
traj.map_property(bin_res=0.25, variable='temp', statistic='mean')

In [ ]:
# Creating a map of the binned mean depth of particles.
traj.map_property(bin_res=0.25, variable='z', statistic='mean')